## 라이브러리 import

In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

### 시드 고정

In [2]:
random_seed = 2021
np.random.seed(random_seed)

## 데이터 불러오기

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# 공통 경로 설정
DATASET_PATH = '/content/drive/MyDrive/Colab Notebooks/timeseries/data'

### train data 만들기 (2020.01.01 ~ 2020.05.24)

In [58]:
train = pd.read_csv(os.path.join(DATASET_PATH, 'train.csv'))  # 2020.01.01 ~ 2020.05.17

In [59]:
valid = pd.read_csv(os.path.join(DATASET_PATH, 'validate.csv'))  # 2020.05.11 ~ 2020.05.24

In [60]:
# trian, valid 합친 다음 겹치는 기간 중복 제거
train_valid = pd.concat([train, valid])
train_valid = train_valid.drop_duplicates()
train_valid

,날짜,시간,10,100,101,120,121,140,150,160,200,201,251,270,300,301,351,352,370,400,450,500,550,600,650,652,1000,1020,1040,1100,1200,1510,2510,3000,4510,5510,6000
0,20200101,0,83247,19128,2611,5161,1588,892,32263,1636,7938,1894,14957,3527,11174,2930,8640,14175,2035,5231,18492,48520,15393,16628,8804,2790,38657,1311,3482,11299,7072,1176,3810,748,3920,2133,3799
1,20200101,1,89309,19027,3337,5502,1650,1043,35609,1644,9896,1967,14682,3420,13884,3062,8349,15654,2417,5483,15283,57904,15834,19785,8483,3381,45759,1162,3849,13180,8771,1283,3763,782,3483,2057,4010
2,20200101,2,66611,14710,2970,4631,1044,921,26821,1104,9343,1838,11120,3587,17141,3255,7025,12520,1859,5058,14327,53876,13866,24501,8167,2670,27425,768,2299,7986,5426,1536,3229,491,2634,1526,3388
3,20200101,3,53290,13753,2270,4242,1021,790,21322,909,10009,1630,9836,3842,20715,2661,7226,9821,1724,4863,12787,51615,13528,28695,9459,2325,18893,632,1716,5703,3156,1104,2882,431,2488,1268,3686
4,20200101,4,52095,17615,2406,3689,1840,922,22711,1354,15497,1264,10668,4559,28308,2894,8201,9038,960,5166,11479,47424,18156,26977,10944,3583,18135,875,2421,5816,2933,1206,2433,499,2952,1927,5608
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
331,20200524,19,314226,98345,10625,28618,8316,6684,141675,6619,23535,8572,67748,16791,33325,19124,42183,63229,13163,35383,73188,100849,84286,26458,15572,10341,139827,8254,16118,23304,14082,8447,21694,2180,15746,10903,21014
332,20200524,20,300001,87871,8226,22706,6981,5743,142933,6295,20953,7851,61609,14119,30216,18610,35556,59070,12788,30675,68255,95107,76441,24140,13340,8872,142469,5225,15297,21919,14526,7332,19732,1990,14096,10028,17787
333,20200524,21,304150,71126,6002,18317,4939,3779,133110,4781,15242,6101,48336,11534,25710,15651,27467,53787,12019,26317,61169,86434,61136,21076,9317,6392,138816,4072,12685,21135,14403,5443,16967,1359,11670,7963,14041
334,20200524,22,236751,44947,3575,11455,3135,2536,98582,3267,8849,4553,33607,8069,18293,9723,16602,38615,8703,17694,47609,58492,37501,12090,5901,3938,100767,2489,8093,14427,10914,3861,11397,859,7270,5194,8230


In [61]:
# 인덱스 정리
train_valid = train_valid.reset_index(drop=True)

In [62]:
# YYYY:MM:DD HH:MM:SS 형식을 가진 '일시' 컬럼 생성
train_valid['일시'] = pd.to_datetime(train_valid['날짜'].astype(str)) + pd.to_timedelta(train_valid['시간'], 'h')

In [63]:
# 날짜 컬럼에서 월, 일, 요일, 주 정보 추출
train_valid['날짜'] = pd.to_datetime(train_valid['날짜'].astype(str))
train_valid['월'] = train_valid['날짜'].dt.month
train_valid['일'] = train_valid['날짜'].dt.day
train_valid['요일'] = train_valid['날짜'].dt.dayofweek
train_valid['주'] = train_valid['날짜'].dt.week

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  


In [64]:
# 공휴일 정보 추가 (공휴일:1, 공휴일아님:0)
train_valid['공휴일'] = 0
train_valid['날짜'] = train_valid['날짜'].astype(str)

for i in range(len(train_valid)):
  if train_valid['날짜'][i] in ['2020-01-01', '2020-01-24', '2020-01-25', '2020-01-26', '2020-01-27', '2020-04-15', '2020-04-30', '2020-05-01', '2020-05-05']:
    train_valid['공휴일'][i] = 1

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [65]:
train_valid

,날짜,시간,10,100,101,120,121,140,150,160,200,201,251,270,300,301,351,352,370,400,450,500,550,600,650,652,1000,1020,1040,1100,1200,1510,2510,3000,4510,5510,6000,일시,월,일,요일,주,공휴일
0,2020-01-01,0,83247,19128,2611,5161,1588,892,32263,1636,7938,1894,14957,3527,11174,2930,8640,14175,2035,5231,18492,48520,15393,16628,8804,2790,38657,1311,3482,11299,7072,1176,3810,748,3920,2133,3799,2020-01-01 00:00:00,1,1,2,1,1
1,2020-01-01,1,89309,19027,3337,5502,1650,1043,35609,1644,9896,1967,14682,3420,13884,3062,8349,15654,2417,5483,15283,57904,15834,19785,8483,3381,45759,1162,3849,13180,8771,1283,3763,782,3483,2057,4010,2020-01-01 01:00:00,1,1,2,1,1
2,2020-01-01,2,66611,14710,2970,4631,1044,921,26821,1104,9343,1838,11120,3587,17141,3255,7025,12520,1859,5058,14327,53876,13866,24501,8167,2670,27425,768,2299,7986,5426,1536,3229,491,2634,1526,3388,2020-01-01 02:00:00,1,1,2,1,1
3,2020-01-01,3,53290,13753,2270,4242,1021,790,21322,909,10009,1630,9836,3842,20715,2661,7226,9821,1724,4863,12787,51615,13528,28695,9459,2325,18893,632,1716,5703,3156,1104,2882,431,2488,1268,3686,2020-01-01 03:00:00,1,1,2,1,1
4,2020-01-01,4,52095,17615,2406,3689,1840,922,22711,1354,15497,1264,10668,4559,28308,2894,8201,9038,960,5166,11479,47424,18156,26977,10944,3583,18135,875,2421,5816,2933,1206,2433,499,2952,1927,5608,2020-01-01 04:00:00,1,1,2,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3442,2020-05-24,19,314226,98345,10625,28618,8316,6684,141675,6619,23535,8572,67748,16791,33325,19124,42183,63229,13163,35383,73188,100849,84286,26458,15572,10341,139827,8254,16118,23304,14082,8447,21694,2180,15746,10903,21014,2020-05-24 19:00:00,5,24,6,21,0
3443,2020-05-24,20,300001,87871,8226,22706,6981,5743,142933,6295,20953,7851,61609,14119,30216,18610,35556,59070,12788,30675,68255,95107,76441,24140,13340,8872,142469,5225,15297,21919,14526,7332,19732,1990,14096,10028,17787,2020-05-24 20:00:00,5,24,6,21,0
3444,2020-05-24,21,304150,71126,6002,18317,4939,3779,133110,4781,15242,6101,48336,11534,25710,15651,27467,53787,12019,26317,61169,86434,61136,21076,9317,6392,138816,4072,12685,21135,14403,5443,16967,1359,11670,7963,14041,2020-05-24 21:00:00,5,24,6,21,0
3445,2020-05-24,22,236751,44947,3575,11455,3135,2536,98582,3267,8849,4553,33607,8069,18293,9723,16602,38615,8703,17694,47609,58492,37501,12090,5901,3938,100767,2489,8093,14427,10914,3861,11397,859,7270,5194,8230,2020-05-24 22:00:00,5,24,6,21,0


### 기온 정보 추가

In [66]:
weather = pd.read_csv(os.path.join(DATASET_PATH, 'weather.csv'), encoding='cp949')
weather

,지점,지점명,일시,기온(°C),기온 QC플래그,강수량(mm),강수량 QC플래그,적설(cm),3시간신적설(cm)
0,108,서울,2020-01-01 00:00,-6.5,NaN,0.0,NaN,NaN,NaN
1,108,서울,2020-01-01 01:00,-5.9,NaN,NaN,9.0,NaN,NaN
2,108,서울,2020-01-01 02:00,-5.7,NaN,NaN,9.0,NaN,NaN
3,108,서울,2020-01-01 03:00,-5.6,NaN,0.0,NaN,NaN,NaN
4,108,서울,2020-01-01 04:00,-5.4,NaN,NaN,9.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...
6955,159,부산,2020-05-24 19:00,19.3,NaN,NaN,NaN,NaN,NaN
6956,159,부산,2020-05-24 20:00,19.7,NaN,NaN,NaN,NaN,NaN
6957,159,부산,2020-05-24 21:00,19.6,NaN,NaN,NaN,NaN,NaN
6958,159,부산,2020-05-24 22:00,19.6,NaN,NaN,NaN,NaN,NaN


In [67]:
# 우선 서울 데이터만 활용
weather = weather[:3480]

In [68]:
# 일시 데이터 형식 통일
weather['일시'] = pd.to_datetime(weather['일시'])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [69]:
# 일시, 기온 정보만 사용
weather_train = weather[['일시', '기온(°C)']]

In [70]:
weather_train

,일시,기온(°C)
0,2020-01-01 00:00:00,-6.5
1,2020-01-01 01:00:00,-5.9
2,2020-01-01 02:00:00,-5.7
3,2020-01-01 03:00:00,-5.6
4,2020-01-01 04:00:00,-5.4
...,...,...
3475,2020-05-24 19:00:00,17.8
3476,2020-05-24 20:00:00,16.7
3477,2020-05-24 21:00:00,15.5
3478,2020-05-24 22:00:00,15.0


In [71]:
train2 = pd.merge(train_valid, weather_train, on='일시', how='left')
train2

,날짜,시간,10,100,101,120,121,140,150,160,200,201,251,270,300,301,351,352,370,400,450,500,550,600,650,652,1000,1020,1040,1100,1200,1510,2510,3000,4510,5510,6000,일시,월,일,요일,주,공휴일,기온(°C)
0,2020-01-01,0,83247,19128,2611,5161,1588,892,32263,1636,7938,1894,14957,3527,11174,2930,8640,14175,2035,5231,18492,48520,15393,16628,8804,2790,38657,1311,3482,11299,7072,1176,3810,748,3920,2133,3799,2020-01-01 00:00:00,1,1,2,1,1,-6.5
1,2020-01-01,1,89309,19027,3337,5502,1650,1043,35609,1644,9896,1967,14682,3420,13884,3062,8349,15654,2417,5483,15283,57904,15834,19785,8483,3381,45759,1162,3849,13180,8771,1283,3763,782,3483,2057,4010,2020-01-01 01:00:00,1,1,2,1,1,-5.9
2,2020-01-01,2,66611,14710,2970,4631,1044,921,26821,1104,9343,1838,11120,3587,17141,3255,7025,12520,1859,5058,14327,53876,13866,24501,8167,2670,27425,768,2299,7986,5426,1536,3229,491,2634,1526,3388,2020-01-01 02:00:00,1,1,2,1,1,-5.7
3,2020-01-01,3,53290,13753,2270,4242,1021,790,21322,909,10009,1630,9836,3842,20715,2661,7226,9821,1724,4863,12787,51615,13528,28695,9459,2325,18893,632,1716,5703,3156,1104,2882,431,2488,1268,3686,2020-01-01 03:00:00,1,1,2,1,1,-5.6
4,2020-01-01,4,52095,17615,2406,3689,1840,922,22711,1354,15497,1264,10668,4559,28308,2894,8201,9038,960,5166,11479,47424,18156,26977,10944,3583,18135,875,2421,5816,2933,1206,2433,499,2952,1927,5608,2020-01-01 04:00:00,1,1,2,1,1,-5.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3442,2020-05-24,19,314226,98345,10625,28618,8316,6684,141675,6619,23535,8572,67748,16791,33325,19124,42183,63229,13163,35383,73188,100849,84286,26458,15572,10341,139827,8254,16118,23304,14082,8447,21694,2180,15746,10903,21014,2020-05-24 19:00:00,5,24,6,21,0,17.8
3443,2020-05-24,20,300001,87871,8226,22706,6981,5743,142933,6295,20953,7851,61609,14119,30216,18610,35556,59070,12788,30675,68255,95107,76441,24140,13340,8872,142469,5225,15297,21919,14526,7332,19732,1990,14096,10028,17787,2020-05-24 20:00:00,5,24,6,21,0,16.7
3444,2020-05-24,21,304150,71126,6002,18317,4939,3779,133110,4781,15242,6101,48336,11534,25710,15651,27467,53787,12019,26317,61169,86434,61136,21076,9317,6392,138816,4072,12685,21135,14403,5443,16967,1359,11670,7963,14041,2020-05-24 21:00:00,5,24,6,21,0,15.5
3445,2020-05-24,22,236751,44947,3575,11455,3135,2536,98582,3267,8849,4553,33607,8069,18293,9723,16602,38615,8703,17694,47609,58492,37501,12090,5901,3938,100767,2489,8093,14427,10914,3861,11397,859,7270,5194,8230,2020-05-24 22:00:00,5,24,6,21,0,15.0


### 기름값 정보 추가

In [72]:
oil = pd.read_csv(os.path.join(DATASET_PATH, 'oilprice.csv'), encoding='cp949')
oil

,번호,지역,상호,주소,기간,상표,셀프여부,고급휘발유,휘발유,경유,실내등유
0,기준 : 일간(20200101~20200531),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,A0006039,서울 강남구,(유)동하석유 힐탑셀프주유소,서울 강남구 논현로 640,20200101.0,SK에너지,셀프,1802.0,1657.0,1495.0,0.0
2,A0006039,서울 강남구,(유)동하석유 힐탑셀프주유소,서울 강남구 논현로 640,20200102.0,SK에너지,셀프,1802.0,1657.0,1495.0,0.0
3,A0006039,서울 강남구,(유)동하석유 힐탑셀프주유소,서울 강남구 논현로 640,20200103.0,SK에너지,셀프,1802.0,1657.0,1495.0,0.0
4,A0006039,서울 강남구,(유)동하석유 힐탑셀프주유소,서울 강남구 논현로 640,20200104.0,SK에너지,셀프,1802.0,1657.0,1495.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
6069,A0001216,서울 강남구,현대오일뱅크㈜직영 오천주유소,서울 강남구 봉은사로 503 (삼성동),20200527.0,SK에너지,일반,2079.0,1789.0,1609.0,959.0
6070,A0001216,서울 강남구,현대오일뱅크㈜직영 오천주유소,서울 강남구 봉은사로 503 (삼성동),20200528.0,SK에너지,일반,2079.0,1789.0,1609.0,959.0
6071,A0001216,서울 강남구,현대오일뱅크㈜직영 오천주유소,서울 강남구 봉은사로 503 (삼성동),20200529.0,SK에너지,일반,2079.0,1789.0,1609.0,1025.0
6072,A0001216,서울 강남구,현대오일뱅크㈜직영 오천주유소,서울 강남구 봉은사로 503 (삼성동),20200530.0,SK에너지,일반,2079.0,1789.0,1609.0,1025.0


In [73]:
# 2020.01.01 ~ 2020.05.24 데이터만 학습에 사용
oil_train = oil.iloc[1:146, :].copy().reset_index(drop=True)

In [74]:
oil_train = oil_train[['기간', '고급휘발유', '휘발유', '경유']]

In [75]:
oil_train['기간'] = oil_train['기간'].astype(int)
oil_train['날짜'] = pd.to_datetime(oil_train['기간'].astype(str))
oil_train = oil_train.drop(columns='기간')
oil_train

,고급휘발유,휘발유,경유,날짜
0,1802.0,1657.0,1495.0,2020-01-01
1,1802.0,1657.0,1495.0,2020-01-02
2,1802.0,1657.0,1495.0,2020-01-03
3,1802.0,1657.0,1495.0,2020-01-04
4,1802.0,1657.0,1495.0,2020-01-05
...,...,...,...,...
140,1565.0,1345.0,1195.0,2020-05-20
141,1565.0,1345.0,1195.0,2020-05-21
142,1565.0,1345.0,1195.0,2020-05-22
143,1565.0,1345.0,1195.0,2020-05-23


In [76]:
train2['날짜'] = pd.to_datetime(train2['날짜'])

In [77]:
train3 = pd.merge(train2, oil_train, on='날짜', how='left')
train3

,날짜,시간,10,100,101,120,121,140,150,160,200,201,251,270,300,301,351,352,370,400,450,500,550,600,650,652,1000,1020,1040,1100,1200,1510,2510,3000,4510,5510,6000,일시,월,일,요일,주,공휴일,기온(°C),고급휘발유,휘발유,경유
0,2020-01-01,0,83247,19128,2611,5161,1588,892,32263,1636,7938,1894,14957,3527,11174,2930,8640,14175,2035,5231,18492,48520,15393,16628,8804,2790,38657,1311,3482,11299,7072,1176,3810,748,3920,2133,3799,2020-01-01 00:00:00,1,1,2,1,1,-6.5,1802.0,1657.0,1495.0
1,2020-01-01,1,89309,19027,3337,5502,1650,1043,35609,1644,9896,1967,14682,3420,13884,3062,8349,15654,2417,5483,15283,57904,15834,19785,8483,3381,45759,1162,3849,13180,8771,1283,3763,782,3483,2057,4010,2020-01-01 01:00:00,1,1,2,1,1,-5.9,1802.0,1657.0,1495.0
2,2020-01-01,2,66611,14710,2970,4631,1044,921,26821,1104,9343,1838,11120,3587,17141,3255,7025,12520,1859,5058,14327,53876,13866,24501,8167,2670,27425,768,2299,7986,5426,1536,3229,491,2634,1526,3388,2020-01-01 02:00:00,1,1,2,1,1,-5.7,1802.0,1657.0,1495.0
3,2020-01-01,3,53290,13753,2270,4242,1021,790,21322,909,10009,1630,9836,3842,20715,2661,7226,9821,1724,4863,12787,51615,13528,28695,9459,2325,18893,632,1716,5703,3156,1104,2882,431,2488,1268,3686,2020-01-01 03:00:00,1,1,2,1,1,-5.6,1802.0,1657.0,1495.0
4,2020-01-01,4,52095,17615,2406,3689,1840,922,22711,1354,15497,1264,10668,4559,28308,2894,8201,9038,960,5166,11479,47424,18156,26977,10944,3583,18135,875,2421,5816,2933,1206,2433,499,2952,1927,5608,2020-01-01 04:00:00,1,1,2,1,1,-5.4,1802.0,1657.0,1495.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3442,2020-05-24,19,314226,98345,10625,28618,8316,6684,141675,6619,23535,8572,67748,16791,33325,19124,42183,63229,13163,35383,73188,100849,84286,26458,15572,10341,139827,8254,16118,23304,14082,8447,21694,2180,15746,10903,21014,2020-05-24 19:00:00,5,24,6,21,0,17.8,1565.0,1345.0,1195.0
3443,2020-05-24,20,300001,87871,8226,22706,6981,5743,142933,6295,20953,7851,61609,14119,30216,18610,35556,59070,12788,30675,68255,95107,76441,24140,13340,8872,142469,5225,15297,21919,14526,7332,19732,1990,14096,10028,17787,2020-05-24 20:00:00,5,24,6,21,0,16.7,1565.0,1345.0,1195.0
3444,2020-05-24,21,304150,71126,6002,18317,4939,3779,133110,4781,15242,6101,48336,11534,25710,15651,27467,53787,12019,26317,61169,86434,61136,21076,9317,6392,138816,4072,12685,21135,14403,5443,16967,1359,11670,7963,14041,2020-05-24 21:00:00,5,24,6,21,0,15.5,1565.0,1345.0,1195.0
3445,2020-05-24,22,236751,44947,3575,11455,3135,2536,98582,3267,8849,4553,33607,8069,18293,9723,16602,38615,8703,17694,47609,58492,37501,12090,5901,3938,100767,2489,8093,14427,10914,3861,11397,859,7270,5194,8230,2020-05-24 22:00:00,5,24,6,21,0,15.0,1565.0,1345.0,1195.0


### test data 만들기 (2020.05.25 ~ 2020.05.31)

In [81]:
test = pd.read_csv(os.path.join(DATASET_PATH, 'test.csv'))
test

,날짜,시간,10,100,101,120,121,140,150,160,200,201,251,270,300,301,351,352,370,400,450,500,550,600,650,652,1000,1020,1040,1100,1200,1510,2510,3000,4510,5510,6000
0,20200518,0,82065,15172,1500,3294,1086,962,28931,1103,3014,1619,12596,3060,7383,2958,5829,11297,2890,5694,19553,18919,9907,2912,1884,1112,28189,618,2790,5147,4331,1329,3665,404,2242,1619,2314
1,20200518,1,51248,9840,813,2356,696,546,17888,720,1839,1502,7741,2132,4790,1743,4296,6781,1752,3651,12454,11849,6292,1750,1118,703,16683,430,1864,3269,2561,921,2081,272,1390,1003,1766
2,20200518,2,39026,7894,760,2413,408,549,13357,498,1360,1233,6475,2098,3965,1345,3957,4762,1250,3199,9640,9392,4961,1104,1004,485,11934,322,1313,2765,1931,920,1764,228,1136,922,1309
3,20200518,3,40993,10137,780,2701,420,741,15544,532,1671,1800,7720,2133,4083,1501,3992,6202,1505,4459,10802,11664,6015,1132,1061,851,13458,326,1766,3320,2060,892,2447,337,1495,975,1912
4,20200518,4,77863,19603,1276,5019,968,1160,32101,968,2574,2384,12018,3502,6430,2882,5957,11987,3176,9054,16490,23479,10676,1945,1520,1420,31698,669,2914,6986,3911,1368,4380,513,2940,1758,3629
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
331,20200531,19,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999
332,20200531,20,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999
333,20200531,21,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999
334,20200531,22,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999


In [82]:
test = test[-168:].copy()

In [83]:
test = test.reset_index(drop=True)

In [84]:
test['일시'] = pd.to_datetime(test['날짜'].astype(str)) + pd.to_timedelta(test['시간'], 'h')

In [85]:
test['날짜'] = pd.to_datetime(test['날짜'].astype(str))
test['월'] = test['날짜'].dt.month
test['일'] = test['날짜'].dt.day
test['요일'] = test['날짜'].dt.dayofweek
test['주'] = test['날짜'].dt.week

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  """


In [86]:
test['공휴일'] = 0
test['날짜'] = test['날짜'].astype(str)

for i in range(len(test)):
  if test['날짜'][i] in ['2020-01-01', '2020-01-24', '2020-01-25', '2020-01-26', '2020-01-27', '2020-04-15', '2020-04-30', '2020-05-01', '2020-05-05']:
    test['공휴일'][i] = 1

In [87]:
test

,날짜,시간,10,100,101,120,121,140,150,160,200,201,251,270,300,301,351,352,370,400,450,500,550,600,650,652,1000,1020,1040,1100,1200,1510,2510,3000,4510,5510,6000,일시,월,일,요일,주,공휴일
0,2020-05-25,0,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,2020-05-25 00:00:00,5,25,0,22,0
1,2020-05-25,1,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,2020-05-25 01:00:00,5,25,0,22,0
2,2020-05-25,2,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,2020-05-25 02:00:00,5,25,0,22,0
3,2020-05-25,3,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,2020-05-25 03:00:00,5,25,0,22,0
4,2020-05-25,4,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,2020-05-25 04:00:00,5,25,0,22,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
163,2020-05-31,19,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,2020-05-31 19:00:00,5,31,6,22,0
164,2020-05-31,20,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,2020-05-31 20:00:00,5,31,6,22,0
165,2020-05-31,21,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,2020-05-31 21:00:00,5,31,6,22,0
166,2020-05-31,22,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,2020-05-31 22:00:00,5,31,6,22,0


### weather 정보 추가
테스트 기간의 데이터는 사용할 수 없으니 그 전 데이터들로 시계열 예측해서 사용

2018.01.01 ~ 2020.05.24 기온 데이터로 2020.05.25 ~ 2020.05.31 기온 예측. Prophet 사용.

In [88]:
w18 = pd.read_csv(os.path.join(DATASET_PATH, 'w2018.csv'), encoding='cp949')
w19 = pd.read_csv(os.path.join(DATASET_PATH, 'w2019.csv'), encoding='cp949')
w20 = pd.read_csv(os.path.join(DATASET_PATH, 'w20.csv'), encoding='cp949')
weather_all = pd.concat([w18, w19, w20])
weather_all

,지점,지점명,일시,기온(°C),강수량(mm),적설(cm),3시간신적설(cm)
0,108,서울,2018-01-01 00:00,-3.2,NaN,NaN,NaN
1,108,서울,2018-01-01 01:00,-3.3,NaN,NaN,NaN
2,108,서울,2018-01-01 02:00,-3.7,NaN,NaN,NaN
3,108,서울,2018-01-01 03:00,-4.0,NaN,NaN,NaN
4,108,서울,2018-01-01 04:00,-4.2,NaN,NaN,NaN
...,...,...,...,...,...,...,...
3643,108,서울,2020-05-31 19:00,20.7,NaN,NaN,NaN
3644,108,서울,2020-05-31 20:00,20.2,NaN,NaN,NaN
3645,108,서울,2020-05-31 21:00,19.8,NaN,NaN,NaN
3646,108,서울,2020-05-31 22:00,19.0,NaN,NaN,NaN


In [89]:
# 2018.01.01 ~ 2020.05.24 데이터
weather_past = weather_all[:-168]

In [90]:
weather_past['일시'] = pd.to_datetime(weather_past['일시'])
weather_past = weather_past[['일시', '기온(°C)']]
weather_past

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,일시,기온(°C)
0,2018-01-01 00:00:00,-3.2
1,2018-01-01 01:00:00,-3.3
2,2018-01-01 02:00:00,-3.7
3,2018-01-01 03:00:00,-4.0
4,2018-01-01 04:00:00,-4.2
...,...,...
3475,2020-05-24 19:00:00,17.8
3476,2020-05-24 20:00:00,16.7
3477,2020-05-24 21:00:00,15.5
3478,2020-05-24 22:00:00,15.0


In [91]:
from fbprophet import Prophet

df = pd.DataFrame()
df['ds'] = weather_past['일시']
df['y'] = weather_past['기온(°C)']

m = Prophet()
m.fit(df)

future = m.make_future_dataframe(freq='H', periods=168)
forecast = m.predict(future)
result = forecast[['ds', 'yhat']]

weather_pred = pd.DataFrame()
weather_pred['일시'] = future
weather_pred['기온(°C)'] = result['yhat']

In [92]:
weather_future = weather_pred[-168:]
weather_future

,일시,기온(°C)
20999,2020-05-25 00:00:00,16.691284
21000,2020-05-25 01:00:00,16.272530
21001,2020-05-25 02:00:00,15.949737
21002,2020-05-25 03:00:00,15.695986
21003,2020-05-25 04:00:00,15.417966
...,...,...
21162,2020-05-31 19:00:00,20.176847
21163,2020-05-31 20:00:00,19.261753
21164,2020-05-31 21:00:00,18.600359
21165,2020-05-31 22:00:00,18.116659


In [93]:
test2 = pd.merge(test, weather_future, on='일시', how='left')
test2

,날짜,시간,10,100,101,120,121,140,150,160,200,201,251,270,300,301,351,352,370,400,450,500,550,600,650,652,1000,1020,1040,1100,1200,1510,2510,3000,4510,5510,6000,일시,월,일,요일,주,공휴일,기온(°C)
0,2020-05-25,0,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,2020-05-25 00:00:00,5,25,0,22,0,16.691284
1,2020-05-25,1,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,2020-05-25 01:00:00,5,25,0,22,0,16.272530
2,2020-05-25,2,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,2020-05-25 02:00:00,5,25,0,22,0,15.949737
3,2020-05-25,3,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,2020-05-25 03:00:00,5,25,0,22,0,15.695986
4,2020-05-25,4,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,2020-05-25 04:00:00,5,25,0,22,0,15.417966
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
163,2020-05-31,19,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,2020-05-31 19:00:00,5,31,6,22,0,20.176847
164,2020-05-31,20,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,2020-05-31 20:00:00,5,31,6,22,0,19.261753
165,2020-05-31,21,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,2020-05-31 21:00:00,5,31,6,22,0,18.600359
166,2020-05-31,22,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,2020-05-31 22:00:00,5,31,6,22,0,18.116659


### 기름값 정보 추가
2020.05.25 ~ 2020.05.31 기름값은 2020.05.24 기름값과 동일하다고 가정 

In [94]:
oil_train

,고급휘발유,휘발유,경유,날짜
0,1802.0,1657.0,1495.0,2020-01-01
1,1802.0,1657.0,1495.0,2020-01-02
2,1802.0,1657.0,1495.0,2020-01-03
3,1802.0,1657.0,1495.0,2020-01-04
4,1802.0,1657.0,1495.0,2020-01-05
...,...,...,...,...
140,1565.0,1345.0,1195.0,2020-05-20
141,1565.0,1345.0,1195.0,2020-05-21
142,1565.0,1345.0,1195.0,2020-05-22
143,1565.0,1345.0,1195.0,2020-05-23


In [95]:
start_date = pd.to_datetime('2020-05-25')
end_date = pd.to_datetime('2020-05-31')

dates = pd.date_range(start_date, end_date, freq='D')

oil_future = pd.DataFrame({'고급휘발유':[1565.0]*7,
                         '휘발유': [1345.0]*7,
                         '경유': [1195.0]*7,
                         '날짜': pd.date_range(start_date,end_date,freq='D'),
})
oil_future

,고급휘발유,휘발유,경유,날짜
0,1565.0,1345.0,1195.0,2020-05-25
1,1565.0,1345.0,1195.0,2020-05-26
2,1565.0,1345.0,1195.0,2020-05-27
3,1565.0,1345.0,1195.0,2020-05-28
4,1565.0,1345.0,1195.0,2020-05-29
5,1565.0,1345.0,1195.0,2020-05-30
6,1565.0,1345.0,1195.0,2020-05-31


In [96]:
test2['날짜'] = pd.to_datetime(test2['날짜'])

In [97]:
test3 = pd.merge(test2, oil_future, on='날짜', how='left')
test3

,날짜,시간,10,100,101,120,121,140,150,160,200,201,251,270,300,301,351,352,370,400,450,500,550,600,650,652,1000,1020,1040,1100,1200,1510,2510,3000,4510,5510,6000,일시,월,일,요일,주,공휴일,기온(°C),고급휘발유,휘발유,경유
0,2020-05-25,0,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,2020-05-25 00:00:00,5,25,0,22,0,16.691284,1565.0,1345.0,1195.0
1,2020-05-25,1,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,2020-05-25 01:00:00,5,25,0,22,0,16.272530,1565.0,1345.0,1195.0
2,2020-05-25,2,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,2020-05-25 02:00:00,5,25,0,22,0,15.949737,1565.0,1345.0,1195.0
3,2020-05-25,3,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,2020-05-25 03:00:00,5,25,0,22,0,15.695986,1565.0,1345.0,1195.0
4,2020-05-25,4,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,2020-05-25 04:00:00,5,25,0,22,0,15.417966,1565.0,1345.0,1195.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
163,2020-05-31,19,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,2020-05-31 19:00:00,5,31,6,22,0,20.176847,1565.0,1345.0,1195.0
164,2020-05-31,20,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,2020-05-31 20:00:00,5,31,6,22,0,19.261753,1565.0,1345.0,1195.0
165,2020-05-31,21,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,2020-05-31 21:00:00,5,31,6,22,0,18.600359,1565.0,1345.0,1195.0
166,2020-05-31,22,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,2020-05-31 22:00:00,5,31,6,22,0,18.116659,1565.0,1345.0,1195.0


## 모델링 & 제출

In [98]:
sub = pd.read_csv(os.path.join(DATASET_PATH, 'sample_submission.csv'))
sub

,timestamp,10,100,101,120,121,140,150,160,200,201,251,270,300,301,351,352,370,400,450,500,550,600,650,652,1000,1020,1040,1100,1200,1510,2510,3000,4510,5510,6000
0,20200525_0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,20200525_1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,20200525_2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,20200525_3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,20200525_4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
163,20200531_19,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
164,20200531_20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
165,20200531_21,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
166,20200531_22,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [55]:
!pip install catboost

     |████████████████████████████████| 76.1 MB 1.3 MB/s 


In [99]:
from catboost import CatBoostRegressor

for road in ['10', '100', '101', '120', '121', '140', '150', '160', '200', '201', '251', '270', '300', '301', '351', '352', '370', '400', '450', '500', '550', '600', '650', '652', '1000', '1020', '1040', '1100', '1200', '1510', '2510', '3000', '4510', '5510', '6000']:
  X_train = train3[['시간', '월', '일', '요일', '주', '공휴일', '기온(°C)', '고급휘발유', '휘발유', '경유']]
  y_train = train3[road]
  X_valid = test3[['시간', '월', '일', '요일', '주', '공휴일', '기온(°C)', '고급휘발유', '휘발유', '경유']]
  y_valid = test3[road]

  cbr = CatBoostRegressor(verbose=200, iterations=3500)
  cbr.fit(X_train, y_train)
  pred = cbr.predict(X_valid)
  sub[road] = pred

Learning rate set to 0.017979
0:	learn: 120043.0400000	total: 2.92ms	remaining: 10.2s
200:	learn: 29271.6893195	total: 360ms	remaining: 5.91s
400:	learn: 21282.0678676	total: 687ms	remaining: 5.31s
600:	learn: 17591.2583093	total: 959ms	remaining: 4.63s
800:	learn: 15406.2264733	total: 1.2s	remaining: 4.04s
1000:	learn: 13797.0465039	total: 1.44s	remaining: 3.58s
1200:	learn: 12539.0550894	total: 1.68s	remaining: 3.22s
1400:	learn: 11507.9272617	total: 1.93s	remaining: 2.89s
1600:	learn: 10707.5405108	total: 2.17s	remaining: 2.57s
1800:	learn: 10039.7471587	total: 2.41s	remaining: 2.27s
2000:	learn: 9460.9375335	total: 2.68s	remaining: 2.01s
2200:	learn: 8968.1030713	total: 2.92s	remaining: 1.72s
2400:	learn: 8574.1453881	total: 3.16s	remaining: 1.45s
2600:	learn: 8205.6695504	total: 3.4s	remaining: 1.18s
2800:	learn: 7894.1201617	total: 3.66s	remaining: 913ms
3000:	learn: 7604.8718919	total: 3.9s	remaining: 648ms
3200:	learn: 7341.0968830	total: 4.13s	remaining: 385ms
3400:	learn: 710

In [101]:
sub.to_csv('oilone.csv', index=0)